In [1]:
import os
os.chdir("../../")#与easymlops同级目录

### 拆分数据

In [2]:
import pandas as pd
data=pd.read_csv("./data/demo.csv")
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
x_train=data[:500]
x_test=data[500:]
y_train=x_train["Survived"]
y_test=x_test["Survived"]
del x_train["Survived"]
del x_test["Survived"]

###  日志记录 
日志通常只需要在生产中使用，所以只在transform_single可用

In [4]:
from easymlops import TablePipeLine
from easymlops.table.preprocessing import *
from easymlops.table.encoding import *

In [5]:
table=TablePipeLine()
table.pipe(FixInput())\
  .pipe(SelectCols(cols=["Age","Fare","Embarked"]))\
  .pipe(TablePipeLine().pipe(TargetEncoding(cols=["Embarked"],y=y_train)))\
  .pipe(FillNa())

x_test_new=table.fit(x_train).transform(x_test)
x_test_new.head(5)

,Age,Fare,Embarked
500,17.0,8.664062,0.334254
501,21.0,7.750000,0.511111
502,0.0,7.628906,0.511111
503,37.0,9.585938,0.334254
504,16.0,86.500000,0.334254


In [6]:
import logging
logger = logging.getLogger("EasyMLOps")
logger.setLevel(logging.INFO)
ch = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [7]:
base_log_info={"user_id":1}

In [8]:
output=table.transform_single({'PassengerId': 1,
 'Cabin': 0,
 'Pclass': 3,
 'Name': 'Braund, Mr. Owen Harris',
 'Sex': 'male',
 'Age': 22.0,
 'SibSp': 1,
 'Parch': 0,
 'Ticket': 'A/5 21171',
 'Fare': 7.25,
 'Embarked': 'S'},logger=logger,log_base_dict=base_log_info)

2023-02-28 15:03:34,536 - EasyMLOps - INFO - {'step': 'step-0', 'pipe_name': <class 'easymlops.table.preprocessing.core.FixInput'>, 'transform': {'PassengerId': 1, 'Pclass': 3, 'Name': 'Braund, Mr. Owen Harris', 'Sex': 'male', 'Age': 22.0, 'SibSp': 1, 'Parch': 0, 'Ticket': 'A/5 21171', 'Fare': 7.25, 'Cabin': '0', 'Embarked': 'S'}, 'user_id': 1}
2023-02-28 15:03:34,536 - EasyMLOps - INFO - {'step': 'step-1', 'pipe_name': <class 'easymlops.table.preprocessing.core.SelectCols'>, 'transform': {'Age': 22.0, 'Fare': 7.25, 'Embarked': 'S'}, 'user_id': 1}
2023-02-28 15:03:34,536 - EasyMLOps - INFO - {'step': 'step-2-0', 'pipe_name': <class 'easymlops.table.encoding.TargetEncoding'>, 'transform': {'Age': 22.0, 'Fare': 7.25, 'Embarked': 0.3342541436464088}, 'user_id': 1}
2023-02-28 15:03:34,536 - EasyMLOps - INFO - {'step': 'step-3', 'pipe_name': <class 'easymlops.table.preprocessing.onevar_operation.FillNa'>, 'transform': {'Age': 22.0, 'Fare': 7.25, 'Embarked': 0.3342541436464088}, 'user_id': 1

In [9]:
output

{'Age': 22.0, 'Fare': 7.25, 'Embarked': 0.3342541436464088}